<a href="https://colab.research.google.com/github/prajwolsubedi/random/blob/main/Rick.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradientai --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.7/410.7 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 9.1 MB/s eta 0:00:00


In [21]:
!pip install python-dotenv

In [23]:
env_content = """
GRADIENT_ACCESS_TOKEN=JJaDJjOZmIrNDgvXyJ6VMy2cjIecGWp4
GRADIENT_WORKSPACE_ID=aa6fce07-26ca-49ab-a4f8-777e45f21aae_workspace
"""

with open("/content/.env", "w") as f:
    f.write(env_content)

In [25]:
import os
from dotenv import load_dotenv
# Load environment variables from .env file located in /content
other_secret = os.getenv("OTHER_SECRET")
os.environ['GRADIENT_ACCESS_TOKEN'] = os.getenv("GRADIENT_ACCESS_TOKEN")
os.environ['GRADIENT_WORKSPACE_ID'] = os.getenv("GRADIENT_WORKSPACE_ID")

In [26]:
gitignore_content = """
.env
*.pyc
__pycache__/
.ipynb_checkpoints/
.config/
"""

with open("/content/.gitignore", "w") as f:
    f.write(gitignore_content)


In [27]:
!cat /content/.gitignore


.env
*.pyc
__pycache__/
.ipynb_checkpoints/
.config/


In [7]:
#for parsing csv source file
import csv

#gradient library
from gradientai import Gradient

path_to_csv_file = "/content/sample_data/Rick.csv"


In [8]:
# find pairs of lines where someone else speaks, then Rick speaks
# append them as two separate lines in rows_to_keep
print("Parsing data...")
rows_to_keep = []
with open(path_to_csv_file, encoding="utf-8-sig") as f:
  reader = csv.DictReader(f, delimiter=",")
  last_row = None
  for row in reader:
    if "Rick" == row["name"] and last_row is not None:
      rows_to_keep.append(last_row)
      rows_to_keep.append(row)
      last_row = None
    else:
      last_row = row

# create a role-playing prompt for training and
# later for prompting
role_play_prompt = "You are Rick Sanchez, a character from the TV show Rick and Morty. You are a brilliant mad scientist who is also cynical, misanthropic, nihilistic, and drinks too much. Respond to the following line of dialog as Rick Sanchez."

# combine pairs of rows from above to
# create prompt + reponse on each line
# using prompt template in 'lines' array
lines = []
for i in range(0, len(rows_to_keep), 2):
  prompt = rows_to_keep[i]
  response = rows_to_keep[i+1]
  start_str = f"<s>### Instruction:\n{role_play_prompt}\n\n###Input:\n"
  prompt = prompt["line"].replace('"','\\"')
  mid_str = '''\n\n### Response:\n'''
  response = response["line"].replace('"','\\"')
  end_str = '''</s>'''
  total_line = start_str + prompt + mid_str + response + end_str
  # each line of training data is a simple object: 'inputs' and actual training string
  obj = {
    "inputs" : total_line
  }
  lines.append(obj)
  # print(total_line) # comment in to see how the formatted lines look
    # these lines could also be written to a jsonl file for use
    # with the command line interface
print(f"Generated {len(lines)} lines to fine-tune")
print(f"Example training line: {lines[0]}")

# split up the lines into manageable chunks
lines_per_chunk = 20
all_chunks = []
for line in lines:
  if len(all_chunks) == 0 or len(all_chunks[-1]) == lines_per_chunk:
    all_chunks.append([])
  all_chunks[-1].append(line)

# fine tune the adapter using the chunks of lines from above
# loop contains a try block to handle network or other
# processing errors gracefully
print(f"\nFine-tuning model adapter")
gradient = Gradient()
base = gradient.get_base_model(base_model_slug="nous-hermes2")
my_adapter = base.create_model_adapter(name="rickbot")
print(f"Created model with ID {my_adapter.id}")
for i in range(len(all_chunks)):
  try:
    print(f"Fine-tuning chunk {i} of {len(all_chunks) - 1}")
    my_adapter.fine_tune(samples=all_chunks[i])
  except Exception as error:
    try:
      error_pieces = str(error).split("\n")
      if len(error_pieces) > 1:
        print(f"*** Error processing chunk {i}: {error_pieces[0]} {error_pieces[1]}")
      else:
        print(f"*** Unknown error on chunk {i}: {error}")
    except KeyboardInterrupt:
      break
    except Exception as inner_error:
      print(inner_error)

Parsing data...
Generated 370 lines to fine-tune
Example training line: {'inputs': '<s>### Instruction:\nYou are Rick Sanchez, a character from the TV show Rick and Morty. You are a brilliant mad scientist who is also cynical, misanthropic, nihilistic, and drinks too much. Respond to the following line of dialog as Rick Sanchez.\n\n###Input:\nWhat, Rick? What’s going on?\n\n### Response:\nI got a surprise for you, Morty.</s>'}

Fine-tuning model adapter
Created model with ID 7276ad1b-2e2e-4901-a274-9c1d721f7f70_model_adapter
Fine-tuning chunk 0 of 18
*** Error processing chunk 0: (422) Reason: unknown
Fine-tuning chunk 1 of 18
Fine-tuning chunk 2 of 18
*** Error processing chunk 2: (422) Reason: unknown
Fine-tuning chunk 3 of 18
Fine-tuning chunk 4 of 18
Fine-tuning chunk 5 of 18
Fine-tuning chunk 6 of 18
*** Error processing chunk 6: (422) Reason: unknown
Fine-tuning chunk 7 of 18
Fine-tuning chunk 8 of 18
Fine-tuning chunk 9 of 18
Fine-tuning chunk 10 of 18
Fine-tuning chunk 11 of 18

In [19]:
role_play_prompt = "You are Rick Sanchez, a character from the TV show Rick and Morty. You are a brilliant mad scientist who is also cynical, misanthropic, nihilistic, and drinks too much. Respond to the following line of dialog as Rick Sanchez."
query = "Guess my name ..."
templated_query = f"<s>### Instruction:\n{role_play_prompt}\n\n###Input:\n{query}\n\n### Response:\n"
response = my_adapter.complete(query=templated_query, max_generated_token_count=500)
print(f"> {query}\n> {response.generated_output}\n\n")

> Guess my name ...
> I'm gonna go with ...




In [20]:
my_adapter.delete()